# GEOJSON Points

    Create Lon Lat points from City, State names
    Imports CSV File with city names
    Uses geoapify to get lon lat

In [2]:
# Dependencies and Setup
import pandas as pd
import requests as req
import time
from geojson import Point, Feature, FeatureCollection, dump

# Import API key
from api_keys import geoapify_key

In [17]:
# Globals
# Note that the master csv file with all of the entries has been 
#  segmented into smaller files. Goal is to use geoapify within
#  their 3000 requests per day limit on free api keys

# The first file to read in - file has blank lon/lat values
raw_uap_data = 'UAP_DATA_4.csv'

# The updated lon/lat file results
updated_uap_data = 'uap_data_output_4.csv'

# The City, State, Coutry group used
grouped_data_requests = 'grouped_data4_output.csv'

# The geojson output
geojson_output = 'uap_data_output4.geojson'

In [11]:
# Load the UAP CSV file into a Pandas DataFrame
uap_data_df = pd.read_csv(raw_uap_data)

# Group city, state, country to limit geoapify queries
cities = uap_data_df.groupby(['City','State','Country']).sum().reset_index()

# Save queries into a list
geoapify_queries = []
geoapify_query = 'https://api.geoapify.com/v1/geocode/search?text='
for colname, data in cities.iterrows():
    city = data.iloc[0]
    State = data.iloc[1]
    Country = data.iloc[2]
    geoapify_queries.append(geoapify_query + 
                            city + ',' + 
                            State + ',' + 
                            Country + '&format=json&apiKey=' + geoapify_key)

print(len(cities))
uap_data_df.sample()

5695


,Occurred,Lon,Lat,City,State,Country,Shape,Summary,Media,Explanation
934,8/22/2004,NaN,NaN,Boise,ID,USA,Oval,Green Oval Shaped UFO Sighted Near Gowen Air F...,NaN,NaN


In [12]:
# limit the for loop as a safety measure
loop_counter = 0
loop_start_at = 3001
loop_end_at = 6000

# Finding the lat lon for the city locations
# Adding / updating lat lon in the uap dataframe
for geo in geoapify_queries:

    try:
        if loop_start_at <= loop_counter <= loop_end_at:  
            # req geoapify
            city_json = req.get(geo).json()

            lon = city_json['results'][0]['lon']
            lat = city_json['results'][0]['lat']
            city_name = city_json['results'][0]['city']
            state_abbr = city_json['results'][0]['state_code']

            # add lon lat to dataframe
            uap_data_df.loc[(uap_data_df['City'] == city_name) & (uap_data_df['State'] == state_abbr), 'Lon'] = lon
            uap_data_df.loc[(uap_data_df['City'] == city_name) & (uap_data_df['State'] == state_abbr), 'Lat'] = lat
            
    except:
        print(city_name)
    
    if loop_counter == loop_end_at:
        break
    loop_counter += 1

    time.sleep(.5)

uap_data_df.sample()

Catlettsburg
Catskill
Cavendish
Cavendish
Cecil Township
Winslow Township
Town of Ellicott
Town of Brookhaven
Centerville
Ceresco
Santa Fe
Chadbourn
La Chalamette
Chance
Chandler
Chandler
Channahon
Chappell Hill
Chardon
Charles City
North Charleston
Charlevoix
Charlotte Amalie
Charlottesville
Berlin
Chaska
Chaska
Chaska
Chattahoochee
Chattahoochee
Chauncey
Town of Dannemora
Cheraw
Cheraw
Cherry
Cherry Hill
Chesaning
Chesapeake
Chesapeake
Chesapeake City
West Pikeland Township
Chesterfield
Chesterfield
Chicago
Chicago Heights
Chicago Heights
Chicago Heights
Chico
Chiefland
Chincoteague
Chiricahua
Summit
Christmas Valley
Christoval
Chuckey
Thousand Oaks
Chupadero
Churchville
Cincinnati
Commerce
Commerce
Clark
Clarksburg
Clarkston
Fort Myers
Clinton Township
Clive
Cloverdale
Coaldale
Coalinga
Coatsburg
Cochise
Cocoa Beach
Cocolalla
Coeur d'Alene
Coeymans
Colbert Heights
Cold Bay
College City
Beavercreek
Cold Spring
Colton
Columbia
Columbia Heights
Columbine
Commerce City
Connellsville
Con

,Occurred,Lon,Lat,City,State,Country,Shape,Summary,Media,Explanation
14310,4/19/2015,NaN,NaN,Caldwell,ID,USA,Light,"Orange lights over Caldwell, ID",NaN,NaN


In [13]:
# Save the data to a csv file
uap_data_df.to_csv(updated_uap_data, index=None)

In [14]:
# save the cities group
cities.to_csv(grouped_data_requests, index=None)

In [18]:
# may want to check the csv file for completeness before running this
# Load the UAP CSV file into a Pandas DataFrame
uap_data_df = pd.read_csv(updated_uap_data)

# Output the geojson data
feature_collections = {}

count = len(uap_data_df)
features = []
for i in range(count):
    testout = uap_data_df.iloc[i]
    point = Point((testout.loc[('Lon')], testout.loc[('Lat')]))

    features.append(Feature(geometry=point, 
                            properties={
                                "Occurred": testout.loc[('Occurred')],
                                "City": testout.loc[('City')],
                                "Country": testout.loc[('Country')],
                                "Shape": testout.loc[('Shape')],
                                "Summary": testout.loc[('Summary')]
                                }))

feature_collection = FeatureCollection(features)


with open(geojson_output, 'w') as f:
    dump(feature_collection, f)
